In [1]:
import rooms.dataset
import render
import torch
import torch.nn as nn
import metrics
import train
import os
import numpy as np
import evaluate
import trace1

import matplotlib.pyplot as plt

/home/gzanin/.local/lib/python3.11/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
dataset_name = "prova"

D = rooms.dataset.dataLoader(dataset_name)

In [ ]:
#training parameters

n_fibonacci = 128 #128 
late_stage_model= "UniformResidual" #"UniformResidual"
toa_perturb = True #True
model_transmission = False #False

skip_train = False #False
continue_train = True #False

n_epochs = 1000 #1000 
batch_size = 4 #4 #4 nel test
lr = 1e-2 #1e-2
pink_noise_supervision = True #True
pink_start_epoch = 500 #500
fs = 48000 #48000 

load_dir= 'precomputed/' + dataset_name
save_dir= '~/prova_training_1000epochs'

skip_inference = False #False
skip_music = False #False
skip_eval = False #False
skip_binaural = False #False

valid = False #False #Evaluate on valid instead of test

In [5]:
R = render.Renderer(n_surfaces=len(D.all_surfaces), n_fibonacci=n_fibonacci,
                        late_stage_model=late_stage_model,
                        toa_perturb = toa_perturb, model_transmission=model_transmission).to(device)

In [6]:
# Use multiple GPUs if available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    R = nn.DataParallel(R).module

In [7]:
# Omnidirectional case
'''
loss_fcn = metrics.training_loss

gt_audio = torch.Tensor(D.RIRs[:, :R.RIR_length])

rendering_method = render.Renderer.render_RIR
'''

In [ ]:
# Directional case

loss_fcn = metrics.training_loss_directional
#loss_fcn = metrics.training_loss_directional_with_decay

for listener_position in D.RIRs:
    for response in listener_position:
        response['t_response'] = torch.Tensor(response['t_response'][:R.RIR_length])
        response['t_response'].to(device) 

gt_audio = D.RIRs
rendering_method = render.Renderer.render_RIR_directional

solo per training le prossime

In [8]:
def initialize(indices, source_xyz, listener_xyzs, surfaces, load_dir):
    Ls = []

    for idx in indices:
        L= render.get_listener(source_xyz=source_xyz, listener_xyz = listener_xyzs[idx], surfaces = surfaces, 
                               load_dir = load_dir, load_num = idx, speed_of_sound = D.speed_of_sound, 
                               max_order = D.max_order, parallel_surface_pairs = D.parallel_surface_pairs, 
                               max_axial_order = D.max_axial_order)
        Ls.append(L)
    return Ls

In [9]:
"""
Training
"""
if not skip_train:
    print("Training")

    #Initialize Listeners
    Ls = initialize(indices=D.train_indices,
                    listener_xyzs=D.xyzs,
                    source_xyz=D.speaker_xyz,
                    surfaces=D.all_surfaces,
                    load_dir=load_dir)
            
    if continue_train:
        R.load_state_dict(torch.load(os.path.join(save_dir,"weights.pt"))['model_state_dict'])

    losses = train.train_loop(R=R, Ls=Ls, train_gt_audio=gt_audio[D.train_indices], D=D,
                        n_epochs = n_epochs, batch_size = batch_size, lr = lr, loss_fcn = loss_fcn,
                        save_dir=save_dir,
                        pink_noise_supervision = pink_noise_supervision,
                        pink_start_epoch=pink_start_epoch,
                        continue_train = continue_train, fs=fs)

else:
    R.load_state_dict(torch.load(os.path.join(save_dir,"weights.pt"))['model_state_dict'])
    R.train = False
    R.toa_perturb = False

Training
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	851
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	859
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	917
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	849
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	814
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	829
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	926
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	864
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	896
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	990
Considered Paths:	59050
Total Considered Paths, after Axial:	59080
Valid Paths:	895
Considered Paths:	59050
Total Considered Paths, after Axial:	59080


/home/gzanin/.local/lib/python3.11/site-packages/torch/functional.py:665: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


loss function calcolata
loss.backward() eseguito
loss:
1.8309056758880615
Omnidirectional case
loss function calcolata
loss.backward() eseguito
loss:
1.901488184928894
Omnidirectional case
loss function calcolata
loss.backward() eseguito
loss:
1.9193669557571411
Omnidirectional case
loss function calcolata
loss.backward() eseguito
loss:
1.8361937999725342
parameters
tensor([[[ 0.0100,  0.0100,  0.0100,  0.0100,  0.0100,  0.0100,  0.0100,
           0.0100,  0.0100, -0.0100],
         [-0.0100, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100,
          -0.0100, -0.0100,  0.0100],
         [-0.0100, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100,
          -0.0100, -0.0100,  0.0100]],

        [[ 0.0100,  0.0100,  0.0100,  0.0100,  0.0100,  0.0100,  0.0100,
           0.0100,  0.0100, -0.0100],
         [-0.0100, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100,
          -0.0100, -0.0100,  0.0100],
         [-0.0100, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100, -0.0100,
 

Inference and evaluation

In [ ]:
pt_file = torch.load(save_dir + '/weights.pt', map_location=device)
R.energy_vector = nn.Parameter(pt_file['model_state_dict']['energy_vector'])
R.source_response = nn.Parameter(pt_file['model_state_dict']['source_response'])
R.directivity_sphere = nn.Parameter(pt_file['model_state_dict']['directivity_sphere'])
R.decay = nn.Parameter(pt_file['model_state_dict']['decay'])
R.RIR_residual = nn.Parameter(pt_file['model_state_dict']['RIR_residual'])
R.spline_values = nn.Parameter(pt_file['model_state_dict']['spline_values'])

R.bp_ord_cut_freqs = nn.Parameter(pt_file['model_state_dict']['bp_ord_cut_freqs'])

In [11]:
#Note - this function relies on precomputed reflection paths
#########above in the original code, I added the case where there are no precomputed paths
def inference(R, source_xyz, xyzs, load_dir, source_axis_1=None, source_axis_2=None):
    """
    Render monoaural RIRs at given precomputed reflection paths.

    Parameters
    ----------
    R: Renderer
        renderer to perform inference on
    source_xyz: np.array (3,)
        3D location of source in meters
    xyzs: np.array (N, 3)
        set of listener locations to render at
    load_dir: str
        directory to load precomputed listener paths
    source_axis_1: np.array (3,)
        first axis specifying virtual source rotation,
        default is None which is (1,0,0)
    source_axis_2: np.array (3,)
        second axis specifying virtual source rotation,
        default is None which is (0,1,0)    

    Returns
    -------
    predictions: np.array (N, T) of predicted RIRs    
    """

    predictions = np.zeros((xyzs.shape[0], R.RIR_length))

    with torch.no_grad():
        R.toa_perturb = False
        for idx in range(xyzs.shape[0]):
            print(idx, flush=True)
            
            if load_dir is None: 
                # Tracing from Scratch
                reflections, transmissions, delays, start_directions, end_directions = (
                    trace1.get_reflections_transmissions_and_delays(
                    source=source_xyz, dest=xyzs[idx], surfaces=D.all_surfaces, speed_of_sound=D.speed_of_sound,
                    max_order=D.max_order,parallel_surface_pairs=D.parallel_surface_pairs, max_axial_order=D.max_axial_order)
                )
                
            else:   
                reflections = np.load(os.path.join(load_dir, "reflections/"+str(idx)+".npy"), allow_pickle=True)
                transmissions = np.load(os.path.join(load_dir, "transmissions/"+str(idx)+".npy"), allow_pickle=True)
                delays = np.load(os.path.join(load_dir, "delays/"+str(idx)+".npy"),allow_pickle=True)
                start_directions = np.load(os.path.join(load_dir, "starts/"+str(idx)+".npy"))

            L = render.ListenerLocation(
                source_xyz=source_xyz,
                listener_xyz=xyzs[idx],
                n_surfaces=R.n_surfaces,
                reflections=reflections,
                transmissions=transmissions,
                delays=delays,
                start_directions = start_directions)

            predict = R.render_RIR(L, source_axis_1=source_axis_1, source_axis_2=source_axis_2)
            predictions[idx] = predict.detach().cpu().numpy()

    return predictions

In [ ]:
"""
Inference, rendering RIR
"""
R.train = False
R.toa_perturb = False
pred_dir = os.path.join(save_dir, "predictions")
if not skip_inference:
    pred_rirs = inference(R=R, source_xyz=D.speaker_xyz, xyzs=D.xyzs, load_dir=load_dir)
    train.makedir_if_needed(pred_dir)
    np.save(os.path.join(pred_dir, "pred_rirs.npy"), pred_rirs)

    if not skip_music:
        pred_music = evaluate.render_music(pred_rirs, D.music_dls)    
        np.save(os.path.join(pred_dir,"pred_musics.npy"), pred_music)
else:
    pred_rirs = np.load(os.path.join(pred_dir, "pred_rirs.npy"))
    pred_music = np.load(os.path.join(pred_dir, "pred_musics.npy"))
    
    
"""
Evaluation of Monoaural Audio Using Metrics
"""
if not skip_eval:
    errors_dir = os.path.join(save_dir, "errors")
    train.makedir_if_needed(errors_dir)
    list_of_metrics = metrics.baseline_metrics

    if valid:
        eval_indices = D.valid_indices
    else:
        eval_indices = D.test_indices

    # Evaluating RIR Interp
    for eval_metric in list_of_metrics:

        metric_name = eval_metric.__name__
        errors = evaluate.compute_error(pred_rirs, gt_audio, metric=eval_metric) #original code
        #errors = evaluate.compute_error(pred_rirs[eval_indices], gt_audio[eval_indices], metric=eval_metric)
        
        np.save(os.path.join(errors_dir, "errors_" + metric_name +".npy"), errors)
        print(metric_name + " Metric:", flush=True)
        print(np.mean(errors[eval_indices]))
        #print(np.mean(errors))
        
    # Evaluating Music Interp
    if not skip_music:
        for eval_metric in list_of_metrics:

            metric_name = eval_metric.__name__

            # Computing Error
            errors_music = evaluate.eval_music(pred_music, D.music, eval_metric)
            #errors_music = evaluate.eval_music(pred_music[eval_indices], D.music[eval_indices], eval_metric)
            
            np.save(os.path.join(errors_dir, "errors_music_" + metric_name +".npy"), errors_music)
            print(metric_name + " Music Metric:", flush=True)
            print(np.mean(errors_music[eval_indices]))
            #print(np.mean(errors_music))

Binaural Rendering

In [ ]:
import binauralize

"""
Binaural Rendering
"""
if not skip_binaural:
    pred_binaural_RIRs = []
    for i in range(D.bin_xyzs.shape[0]):
        binaural_RIR_xyz = D.bin_xyzs[i]
        bin_rir = binauralize.render_binaural(R=R, source_xyz = D.speaker_xyz,
                                            source_axis_1=None, source_axis_2=None,
                                            listener_xyz=binaural_RIR_xyz,
                                            listener_forward=D.default_binaural_listener_forward, 
                                            listener_left=D.default_binaural_listener_left,
                                            surfaces=D.all_surfaces,
                                            speed_of_sound=D.speed_of_sound,
                                            parallel_surface_pairs=D.parallel_surface_pairs,
                                            max_order=D.max_order, max_axial_order=D.max_axial_order)

        pred_binaural_RIRs.append(bin_rir)

    pred_binaural_RIRs = np.array(pred_binaural_RIRs)
    np.save(os.path.join(pred_dir, "pred_bin_RIRs.npy"), pred_binaural_RIRs)

    if not skip_music:
        pred_L = pred_binaural_RIRs[:,0,:]
        pred_R = pred_binaural_RIRs[:,1,:]

        pred_L_music = evaluate.render_music(pred_L, D.music_dls[:pred_L.shape[0]])
        pred_R_music = evaluate.render_music(pred_R, D.music_dls[:pred_R.shape[0]])

        pred_bin_music = np.stack((pred_L_music, pred_R_music), axis=2)
        print(pred_bin_music.shape)
        np.save(os.path.join(pred_dir, "pred_bin_musics.npy"), pred_bin_music)

prova di ascolto

In [ ]:
listener_1 = render.get_listener(source_xyz= np.array([5,3,1.5]), listener_xyz = np.array([5,7,1.5]), surfaces = D.all_surfaces, 
                               load_dir = load_dir, load_num = None, speed_of_sound = D.speed_of_sound, 
                               max_order = D.max_order, parallel_surface_pairs = D.parallel_surface_pairs, 
                               max_axial_order = D.max_axial_order)

listener_2 = render.get_listener(source_xyz= np.array([9.9,9.9,2.9]), listener_xyz = np.array([5,3,1.5]), surfaces = D.all_surfaces, 
                               load_dir = load_dir, load_num = None, speed_of_sound = D.speed_of_sound, 
                               max_order = D.max_order, parallel_surface_pairs = D.parallel_surface_pairs, 
                               max_axial_order = D.max_axial_order)

In [10]:
RIR_1 = R.render_RIR(listener_1)

In [11]:
RIR_2 = R.render_RIR(listener_2)

In [ ]:
# RIR_1 plot
plt.plot(RIR_1.detach().cpu())
plt.title("RIR_1")
plt.xlabel("Sample")
plt.ylabel("Value")
plt.grid(True)
plt.show()

In [ ]:
# RIR_2" plot
plt.plot(RIR_2.detach().cpu())
plt.title("Plot")
plt.xlabel("Indice")
plt.ylabel("Valore")
plt.grid(True)
plt.show()

In [14]:
predicted_music_1 = evaluate.render_music(np.array([RIR_1.detach().cpu()]), np.array([D.music_dls[0]]), device = device)
predicted_music_2 = evaluate.render_music(np.array([RIR_2.detach().cpu()]), np.array([D.music_dls[0]]), device = device)

In [15]:
import sounddevice as sd

duration = predicted_music_1.shape[2]/fs  # Duration in seconds
t = np.linspace(0, duration, int(fs * duration), endpoint=False)


sd.play(predicted_music_1[0][0], samplerate=48000)
sd.wait()  


In [16]:
duration = predicted_music_2.shape[2]/fs 
t = np.linspace(0, duration, int(fs * duration), endpoint=False)


sd.play(predicted_music_2[0][0], samplerate=48000)
sd.wait() 

In [16]:
sd.play(D.music_dls[9][0], samplerate=48000)
sd.wait() 